## Week 6: DNA Methylation
### Radhika Jangi 10/16/2020

### 1. Getting data
SRR3083926_1.chr6.fastq from STEM-seq E4.0ICM rep1<br/>
SRR3083926_2.chr6.fastq from STEM-seq E4.0ICM rep1<br/>
SRR3083929_1.chr6.fastq from STEM-seq E5.5Epi rep1<br/>
SRR3083929_2.chr6.fastq from STEM-seq E5.5Epi rep1

bismark_genome_preparation --bowtie2 chr6<br/>

bismark --genome chr6/ -1 SRR3083926_1.chr6.fastq,SRR3083929_1.chr6.fastq -2 SRR3083926_2.chr6.fastq,SRR3083929_2.chr6.fastq<br/>

samtools sort SRR3083926_1.chr6_bismark_bt2_pe.bam -o SRR26.sorted.bam<br/>
samtools sort SRR3083929_1.chr6_bismark_bt2_pe.bam -o SRR29.sorted.bam<br/>

samtools index SRR26.sorted.bam<br/>
samtools index SRR29.sorted.bam<br/>

In [1]:
f = open('SRR3083926_1.chr6_bismark_bt2_pe.bedGraph', 'r')
SRR26 = f.readlines()
SRR26 = SRR26[1:]
f.close()
fs = open('SRR3083929_1.chr6_bismark_bt2_pe.bedGraph', 'r')
SRR29 = fs.readlines()
SRR29 = SRR29[1:]
fs.close()

In [2]:
SRR26[0].split()

['chr6', '49999926', '49999927', '0']

In [3]:
f = open('mm10_refseq_genes_chr6_50M_60M.bed', 'r')
refseq = f.readlines()
f.close()

In [4]:
# Keeps unique genes and their coordinates
unique_genes = {}
for line in refseq:
    gene_coord = list(map(int,line.split()[5:7]))
    gene_name = line.split()[12]
    if gene_name not in unique_genes:
        unique_genes[gene_name] = gene_coord

In [6]:
def methylation_count(srr, search_range):
    meth_score = 0
    count = 0
    for line in srr:
        meth_range = list(map(int,line.split()[1:3])
        score = float(line.split()[3])
        if meth_range[0]>=search_range[1] and meth_range[1]<=search_range[0]:
            count+=1
            meth_score+= score
    return meth_score/count

SyntaxError: invalid syntax (<ipython-input-6-4bfda9e2836d>, line 6)

In [ ]:
# Maps genes to fold change from e4 to e5.5
genes_mfold = {}
for gene, coords in unique_genes.items():
    e4 = methylation_count(SRR26,coords)
    print(e4)
    e5_5 = methylation_count(SRR29,coords)
    if e4>0:
        genes_mfold[gene] = (e5_5-e4)/e4
        print("Gene: {}\tFold change: {}".format(gene,(e5_5-e4)/e4))
#print(genes_mfold)